In [ ]:
import pandas as pd

In [ ]:
# type: ignore
generations = pd.read_excel(
    "/Users/vadmin/Downloads/Hourly Settlement Oct 2025 Update 4.xlsx",
    sheet_name="Generation",
    usecols=["Date", "Time", "Generation", "GMeter"],
)

name_mapping = pd.read_excel(
    "/Users/vadmin/Downloads/Hourly Settlement Oct 2025 Update 4.xlsx",
    sheet_name="Generation_Register",
    usecols=["GMeter", "Generator_Name"],
)

generations = generations.dropna(subset=["Date", "Time", "Generation", "GMeter"]).copy()
generations["Datetime"] = pd.to_datetime(
    generations["Date"].astype(str) + " " + generations["Time"].astype(str)
)
generations = generations.drop(columns=["Date", "Time"]).rename(
    columns={"GMeter": "Plant"}
)

plant_name_dict = dict(zip(name_mapping["GMeter"], name_mapping["Generator_Name"]))
generations["Plant"] = generations["Plant"].map(plant_name_dict)
generations.head(3)

In [ ]:
# type: ignore
consumptions = pd.read_excel(
    "/Users/vadmin/Downloads/Hourly Settlement Oct 2025 Update 4.xlsx",
    sheet_name="Load_Consumption",
    usecols=["Day", "Time", "Consumption", "CMeter"],
)

name_mapping = pd.read_excel(
    "/Users/vadmin/Downloads/Hourly Settlement Oct 2025 Update 4.xlsx",
    sheet_name="Load_Register",
    usecols=["Meter No.", "Customer"],
)

consumptions = consumptions.dropna(
    subset=["Day", "Time", "Consumption", "CMeter"]
).copy()
consumptions["Datetime"] = pd.to_datetime(
    consumptions["Day"].astype(str) + " " + consumptions["Time"].astype(str)
)
consumptions = consumptions.drop(columns=["Day", "Time"]).rename(
    columns={"CMeter": "Consumer"}
)

plant_name_dict = dict(zip(name_mapping["Meter No."], name_mapping["Customer"]))
consumptions["Consumer"] = consumptions["Consumer"].map(plant_name_dict)
consumptions.head(3)

In [ ]:
# type: ignore
plant_consumer = pd.read_excel(
    "/Users/vadmin/Downloads/Hourly Settlement Oct 2025 Update 4.xlsx",
    sheet_name="Contract_Register",
    usecols=["Wholesale_Supplier", "Load", "EnergyShared%"],
)
plant_consumer.rename(
    columns={"Wholesale_Supplier": "Plant", "Load": "Consumer", "EnergyShared%": "Pct"},
    inplace=True,
)
plant_consumer.head(3)

In [ ]:
# type: ignore
generations.to_parquet("generations.parquet")
consumptions.to_parquet("actual_consumptions.parquet")
plant_consumer.to_parquet("plant_consumer.parquet")

In [ ]:
# type: ignore
generations.to_parquet("generations.parquet")
consumptions.to_parquet("actual_consumptions.parquet")
plant_consumer.to_parquet("plant_consumer.parquet")

In [21]:
import pandas as pd

generations = pd.read_excel(
    "/Users/vadmin/Downloads/SettlementUploadfile.xlsx",
    sheet_name="Generation",
    usecols=["Date", "Time", "Generation", "Gen_Consumption", "GMeter"],
)

name_mapping_gen = pd.read_excel(
    "/Users/vadmin/Downloads/SettlementUploadfile.xlsx",
    sheet_name="Generation_Register",
    usecols=["GMeter", "Generator_Name"],
)
generations = generations.dropna(
    subset=["Date", "Time", "Generation", "Gen_Consumption", "GMeter"]
).copy()
generations["Datetime"] = pd.to_datetime(
    generations["Date"].astype(str) + " " + generations["Time"].astype(str)
)
generations = generations.drop(columns=["Date", "Time"]).rename(
    columns={"GMeter": "Plant"}
)
plant_name_dict = dict(
    zip(name_mapping_gen["GMeter"], name_mapping_gen["Generator_Name"])
)
generations["Plant"] = generations["Plant"].map(plant_name_dict)
generations = (
    generations.set_index("Datetime")
    .groupby("Plant")
    .resample("h", include_groups=False)["Generation", "Gen_Consumption"]
    .sum()
    .reset_index()
)


In [22]:
generations

,Plant,Datetime,Generation,Gen_Consumption
0,Akosombo,2025-08-01 00:00:00,293880700.0,0.0
1,Akosombo,2025-08-01 01:00:00,546573300.0,0.0
2,Akosombo,2025-08-01 02:00:00,550394100.0,0.0
3,Akosombo,2025-08-01 03:00:00,542032100.0,0.0
4,Akosombo,2025-08-01 04:00:00,547399000.0,0.0
...,...,...,...,...
16385,TTIPP,2025-08-31 20:00:00,106266700.0,0.0
16386,TTIPP,2025-08-31 21:00:00,106390400.0,0.0
16387,TTIPP,2025-08-31 22:00:00,106539900.0,0.0
16388,TTIPP,2025-08-31 23:00:00,106558000.0,0.0


,Plant,Datetime,Generation
0,Akosombo,2025-08-01 00:00:00,293880700.0
1,Akosombo,2025-08-01 01:00:00,546573300.0
2,Akosombo,2025-08-01 02:00:00,550394100.0
3,Akosombo,2025-08-01 03:00:00,542032100.0
4,Akosombo,2025-08-01 04:00:00,547399000.0
...,...,...,...
16385,TTIPP,2025-08-31 20:00:00,106266700.0
16386,TTIPP,2025-08-31 21:00:00,106390400.0
16387,TTIPP,2025-08-31 22:00:00,106539900.0
16388,TTIPP,2025-08-31 23:00:00,106558000.0
